In [1]:
import numpy as np
import xarray as xr
import glob 
from scipy import stats
import random
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

def CC_map(slp_map, timeseries):
    """
    This function creates a one-pt correlation coefficient map between slp and a given timeseries.
    """
    CC = []
    for lat in range(np.shape(slp_map)[1]):
        lat_line = []
        for lon in range(np.shape(slp_map)[2]):
            slp_ts = slp_map[:,lat,lon]
            correlation = stats.pearsonr(slp_ts, timeseries)[0]
            lat_line.append(correlation)
        CC.append(lat_line)
    return(CC)

def dynamic_adjustment(X, Y, weights, pls_number):
    # find timeperiod
    decades = np.arange(195.8, 201.5, 0.1)
    
    # detrend X and Y
    #Y_detrended = detrend_Y(Y)
    #X_detrended = detrend_X(X)  
    
    # find correlation-coefficient map (CC)
    CC = CC_map(X, Y)

    # weight CC by area and normalize
    CC_weighted = np.multiply(CC, weights[:,np.newaxis])
    CC_weighted_1d = CC_weighted.ravel()
    CC_normed = CC_weighted_1d/np.sqrt(np.dot(CC_weighted_1d, CC_weighted_1d))
    CC_weighted_normed = np.reshape(CC_normed, np.shape(CC_weighted))

    # project X onto weighted CC
    Z = np.array([np.dot(CC_weighted_normed.ravel(), X[i].ravel()) for i in range(0, len(X))])
    Z_normed =  Z/np.sqrt(np.dot(Z, Z))
    X_sample_x_structure = np.reshape(X, (len(X), np.shape(X)[1]*np.shape(X)[2]))
    P = np.matmul(X_sample_x_structure.T, Z_normed)

    # regress Z out of Y
    beta = np.matmul(Z_normed.T, Y)
    Y_regressed_out_Z = Y - beta*Z_normed # should this Z be normed?

    # regress Z out of X
    X_regressed_out_Z = X_sample_x_structure - np.matmul(Z_normed.reshape(-1,1), P.T.reshape(1,-1))
    X_regressed_out_Z = X_regressed_out_Z.reshape(np.shape(X))
    
    return(X_regressed_out_Z, Y_regressed_out_Z, Z_normed)

def detrend_Y(Y):
    # detrend Y
    y_times = np.arange(0, len(Y))
    Y_regression = stats.linregress(x=y_times, y=Y)
    Y_regression = Y_regression[1] + Y_regression[0]*y_times
    Y_detrended = Y - Y_regression
    return(Y_detrended)

def detrend_X(X):
    # detrend X
    x_times = np.arange(0, len(X))
    X_detrended = []
    for lat in range(np.shape(X)[1]):
        lat_line = []
        for lon in range(np.shape(X)[2]):
            X_ts = X[:,lat,lon]
            X_regression = stats.linregress(x=x_times, y=X_ts)
            X_regression = X_regression[1] + X_regression[0]*x_times
            X_detrended_ts = X_ts - X_regression
            lat_line.append(X_detrended_ts)
        X_detrended.append(lat_line)
    X_detrended = np.swapaxes(X_detrended, 0,2)
    X_detrended = np.swapaxes(X_detrended, 1,2)  
    return(X_detrended)

# load and open processed data
filename = '/home/disk/pna2/aodhan/CESM2_LENS/NDJFM_TASandPSL_Maps.nc'
NDJFM_data = xr.open_dataset(filename)

In [4]:
NDJFM_data

<xarray.Dataset>
Dimensions:          (ensemble_member: 100, period: 40, time: 44, lat: 72, lon: 144)
Coordinates:
  * ensemble_member  (ensemble_member) float64 1.0 2.0 3.0 ... 98.0 99.0 100.0
  * period           (period) float64 1.859e+03 1.864e+03 ... 2.054e+03
  * time             (time) float64 1.0 2.0 3.0 4.0 5.0 ... 41.0 42.0 43.0 44.0
  * lat              (lat) float64 -88.75 -86.25 -83.75 ... 83.75 86.25 88.75
  * lon              (lon) float64 1.25 3.75 6.25 8.75 ... 353.8 356.2 358.8
Data variables:
    TAS              (ensemble_member, period, time, lat, lon) float32 ...
    PSL              (ensemble_member, period, time, lat, lon) float32 ...
    ArcticTAS        (ensemble_member, period, time) float32 ...

In [3]:
for idx in range(100):
    print(idx)
    xarray = NDJFM_data.isel(ensemble_member = idx)
    PSL_fields_timeslice = []
    TAS_fields_timeslice = []
    tas_timeseries_timeslice = []
    print(xarray)
    for timeperiod in range(0, xarray.period.size):
        raw_timeseries = xarray.isel(period = timeperiod)
        # center and standardize data for dynamic adjustment
        # Do arctic timeseries first
        Arctic_TAS = raw_timeseries.ArcticTAS.values
        Arctic_TAS_MeanRemoved = Arctic_TAS - np.nanmean(Arctic_TAS)
        Arctic_TAS_stdzd = Arctic_TAS_MeanRemoved/np.nanstd(Arctic_TAS_MeanRemoved)

        # Now do PSL fields
        PSL_fields = raw_timeseries.PSL.values
        PSL_fields_MeanRemoved = PSL_fields - np.nanmean(PSL_fields, axis=0)
        PSL_fields_stdzd = np.divide(PSL_fields_MeanRemoved, np.nanstd(PSL_fields_MeanRemoved, axis=0)[np.newaxis,:,:])
        PSL_fields_stdzd_NH = PSL_fields_stdzd[:,44:]
        
        # Now do TAS fields
        TAS_fields = raw_timeseries.TAS.values
        TAS_fields_MeanRemoved = TAS_fields - np.nanmean(TAS_fields, axis=0)
        TAS_fields_stdzd = np.divide(TAS_fields_MeanRemoved, np.nanstd(TAS_fields_MeanRemoved, axis=0)[np.newaxis,:,:])

        # Dynamically Adjust
        X = PSL_fields_stdzd_NH.copy()
        Y = Arctic_TAS_stdzd.copy()
        slp_weights = np.cos(np.deg2rad(raw_timeseries.lat.values[44:]))
        Zs_tas = []
        n_passes = 3
        for k in range(n_passes):            
            # set X, Y and Z to outputs of the dynamic adjustment
            X, Y, Z = dynamic_adjustment(X, Y, weights=slp_weights, pls_number=k)
            Zs_tas.append(Z)

        # regress components of dynamic adjustment back onto timeseries
        tas_reg = LinearRegression().fit(np.transpose(Zs_tas), Arctic_TAS_MeanRemoved)
        tas_dynamically_induced = tas_reg.predict(np.transpose(Zs_tas))
        tas_thermodynamically_induced = Arctic_TAS_MeanRemoved - tas_dynamically_induced

        # append data for training
        PSL_fields_timeslice.append(PSL_fields)
        TAS_fields_timeslice.append(TAS_fields)
        tas_timeseries_timeslice.append([Arctic_TAS, tas_dynamically_induced, tas_thermodynamically_induced])
    
    # create some data with different dimensions
    ensemble_number = idx
    period = xarray.period.values
    years = raw_timeseries.time.values
    lon = raw_timeseries.lon.values
    lat = raw_timeseries.lat.values
    RealizedDynamicThermodynamic = [0,-1,1]

    # create a new xarray dataset with dimensions
    ds = xr.Dataset({
        'PSL': (['period', 'years', 'lat', 'lon'], PSL_fields_timeslice),
        'TAS': (['period', 'years', 'lat', 'lon'], TAS_fields_timeslice),
        'Arctic_TAS': (['period', 'RealizedDynamicThermodynamic', 'years'], tas_timeseries_timeslice),
        'period': period,
        'lat': lat,
        'lon': lon,
        'years': years,
        'RealizedDynamicThermodynamic': RealizedDynamicThermodynamic, 
    })

    # save the dataset to a netCDF file
    ds.to_netcdf('/home/disk/pna2/aodhan/CESM2_LENS/DynamicallySeparated/Ensemble_{ensemble_member}.nc'.format(ensemble_member=ensemble_number+1))
    ds.close()


0
<xarray.Dataset>
Dimensions:          (period: 40, time: 44, lat: 72, lon: 144)
Coordinates:
    ensemble_member  float64 1.0
  * period           (period) float64 1.859e+03 1.864e+03 ... 2.054e+03
  * time             (time) float64 1.0 2.0 3.0 4.0 5.0 ... 41.0 42.0 43.0 44.0
  * lat              (lat) float64 -88.75 -86.25 -83.75 ... 83.75 86.25 88.75
  * lon              (lon) float64 1.25 3.75 6.25 8.75 ... 353.8 356.2 358.8
Data variables:
    TAS              (period, time, lat, lon) float32 ...
    PSL              (period, time, lat, lon) float32 ...
    ArcticTAS        (period, time) float32 ...
